In [66]:
# Instalar DVC
!pip install dvc

In [67]:
# Para llevar el registro
!pip install comet_ml
import comet_ml
comet_ml.init(project_name = 'Tarea_4_CatsDogs')

In [68]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

In [69]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Prueba 02')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/tarea-4-catsdogs/088caac2aa014244a7f63f8cc082aecb



In [70]:
# Dimensiones de las imágenes
image_w = 150
image_h = 150
input_shape = (image_w, image_h, 3)

# Directorio
train_dir='/kaggle/input/dogscats/train'
test_dir='/kaggle/input/dogscats/test'

In [71]:
# Parámetros
parameters = {
    'num_class' : 2,
    'epochs' : 30,
    'batch_size' : 100,
    'learning_rate' : 0.0001,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'Adam',
    'num_train' : 20000,
    'num_test' : 5000,
}
experiment.log_parameters(parameters)

In [72]:
# Solo toma el entero
epoch_steps = parameters['num_train'] // parameters['batch_size']
test_steps = parameters['num_test'] // parameters['batch_size']

In [73]:
# Se preparan las imágenes
gentrain = ImageDataGenerator(rescale = 1./255.)                                   # Rescala entre 0 y 1 cada canal
train = gentrain.flow_from_directory(train_dir,                                    # Directorio donde se leen las imágenes
                                   batch_size = parameters['batch_size'],          
                                   target_size = (image_w,image_h),                # Redimensiona las imágenes
                                   class_mode = 'binary')                          # Tipo de clasificación

gentest = ImageDataGenerator(rescale = 1./255.)
test = gentest.flow_from_directory(test_dir,
                                 target_size=(image_w,image_h),
                                 batch_size=parameters['batch_size'],
                                 class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [74]:
# Arquitectura
model = Sequential()

model.add(Conv2D(10,(3,3), input_shape = (input_shape)))    # 10 mapas de características | ventana de atención de 3x3
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(20, (3,3), kernel_regularizer = l2(0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(40, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(20, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(10, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

          
model.add(Flatten())
          
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dropout(0.1))          

model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dropout(0.2))
          
model.add(Dense(1))
model.add(Activation('sigmoid'))
          
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 148, 148, 10)      280       
                                                                 
 activation_38 (Activation)  (None, 148, 148, 10)      0         
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 74, 74, 10)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 72, 72, 20)        1820      
                                                                 
 activation_39 (Activation)  (None, 72, 72, 20)        0         
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 36, 36, 20)       0         
 g2D)                                                 

In [75]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.hdf5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min'
)

In [76]:
# Compilación y entrenamiento
model.compile(
    loss = parameters['loss'],
    optimizer='Adam',
    metrics=['accuracy']
)
model.fit(
    train,
    batch_size = parameters['batch_size'],
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = test,
    steps_per_epoch = epoch_steps,
    validation_steps = test_steps,
    callbacks = [checkpoint]
)


Epoch 1/30
200/200 [==============================] - ETA: 0s - loss: 0.7093 - accuracy: 0.5514
Epoch 1: val_loss improved from inf to 0.63772, saving model to best_val_loss.hdf5
200/200 [==============================] - 80s 374ms/step - loss: 0.7093 - accuracy: 0.5514 - val_loss: 0.6377 - val_accuracy: 0.6456
Epoch 2/30
200/200 [==============================] - ETA: 0s - loss: 0.6199 - accuracy: 0.6639
Epoch 2: val_loss improved from 0.63772 to 0.60254, saving model to best_val_loss.hdf5
200/200 [==============================] - 48s 241ms/step - loss: 0.6199 - accuracy: 0.6639 - val_loss: 0.6025 - val_accuracy: 0.6858
Epoch 3/30
200/200 [==============================] - ETA: 0s - loss: 0.5745 - accuracy: 0.7131
Epoch 3: val_loss improved from 0.60254 to 0.53711, saving model to best_val_loss.hdf5
200/200 [==============================] - 49s 243ms/step - loss: 0.5745 - accuracy: 0.7131 - val_loss: 0.5371 - val_accuracy: 0.7346
Epoch 4/30
200/200 [==============================] -

In [77]:
experiment.log_model("CatsDogs", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-4-catsdogs/088caac2aa014244a7f63f8cc082aecb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                 : (0.5514000058174133, 0.899399995803833)
COMET INFO:     batch_accuracy [600]          : (0.46000000834465027, 0.9599999785423279)
COMET INFO:     batch_loss [600]              : (0.1704884022474289, 0.8273438215255737)
COMET INFO:     epoch_duration [30]           : (47.687705196000024, 80.08572785700017)
COMET INFO:     loss [30]                     : (0.26355883479118347, 0.7093039751052856)
COMET INFO:     val_accuracy [30]             : (0.6456000208854675, 0.85379999876022